In [12]:
#### run - 1st cell
import h5py
from py4xs.hdf import lsh5
import cv2
import pylab as plt
import numpy as np
import PIL 
import json
import os
os.chdir('/Users/bashit.a/Documents/Alzheimer/Nov-2021/ROIs')     # location where your h5 and exp file are    Mar-24/BNL-Data      /Dec-2020/

In [13]:
#### run - 2nd cell (make changes accordingly)
pixel_per_mm = 592   # July-2021 --> 620 (190 increment) March-2021 525 (160 increment)  Nov-2021 --> 592 (180 increment)

In [4]:
fh5 = h5py.File("P1img.h5", "r")
sn = lsh5(fh5, top_only=True, print_attrs=False)[0]
grp = fh5[f"{sn}/primary/data"]
sx_pos = grp["ss_sx"][...]
sz_pos = grp["ss_sz"][...]
grp["camSol_image"].shape

OSError: Unable to open file (unable to open file: name = 'P1img.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
[sx_pos.max(), sz_pos.max()]

[5.9999970000000005, 12.500307999999999]

In [5]:
imgs = [PIL.Image.fromarray(cv2.cvtColor(grp['camSol_image'][i,0], cv2.COLOR_BGR2GRAY))
        for i in range(grp['camSol_image'].shape[0])]

NameError: name 'grp' is not defined

In [6]:
header = json.loads(fh5[sn].attrs['start'])
nx,nz = header['shape']
step_size = 1.5
img_size = imgs[0].size

In [7]:
imgt = PIL.Image.new(imgs[0].mode, (int((nx-1)*step_size*pixel_per_mm)+img_size[0], 
                                    int((nz-1)*step_size*pixel_per_mm)+img_size[1]))

In [8]:
# this has to do with the image orientation and how the motor motion is perceived by the cameras
for i in range(len(sx_pos)-1):
#    imgt.paste(imgs[i+1].crop((0, 0, int(pixel_per_mm*1.5), int(pixel_per_mm*1.5))), 
    #im2 = cv2.morphologyEx(np.asarray(imgs[i+1]), cv2.MORPH_TOPHAT, kernel100)
    imgt.paste(imgs[i+1], #PIL.Image.fromarray(im2), # 
               (int((sx_pos.max()-sx_pos[i])*pixel_per_mm), 
                int((sz_pos.max()-sz_pos[i])*pixel_per_mm)))

In [9]:
imgt.save(f"{sn}.png")

In [3]:
#### run - 3rd cell
from PIL import Image,ImageDraw

def annotate_img(fn, sdict):
    # fn1 is gray scale png

    im0 = Image.open(fn).convert("RGB")
    dr = ImageDraw.Draw(im0)

    for sn in sdict.keys():
        plist = sdict[sn]
        for i in range(len(plist)):
            dr.rectangle(plist[i], outline="red", width=3)
    
    froot,ext = fn.rsplit(".", 1)
    im0.convert("P").save(f"{froot}-ROIs.{ext}")

In [30]:
#### run - 6th cell
#### Nov-2021 run

sdict5 = {"2512_EC": [[8354, 2418, 8714, 2778], [8714, 2418, 9074, 2778], [8354, 2778, 8714, 3138], [8714, 2778, 9074, 3138] ], "2512_EC_mica" : [[7900, 2200, 7915, 2215]]}

sdict6 = {"2029_Dentate_gyrus": [[770, 1960, 950, 2500],[950, 1780, 1490, 1960], [950, 1960, 1490, 2140], 
                                 [950, 2140, 1490, 2320], [950, 2320, 1490, 2500]], 
          "2029_dentate_gyrus_mica":  [[1946, 2416, 1961, 2431]] ,
          "2346_Dentate_gyrus" :  [[5534, 2637, 5894, 2817], [5688, 2220, 6048, 2400], [6275, 2750, 6635, 2930]], 
          "2346_dentate_gyrus_mica" :  [[5634, 3189, 5649, 3204]] ,
          "sample6_bkg": [[3982, 2926, 3997, 2941]]}
# annotate_img("holder5_img.png", sdict5) # {**sdict3, **sdict4})   
annotate_img("holder6_img.png", sdict6) # {**sdict3, **sdict4})   

In [14]:
#### run - 4th cell
## Region calculation

import math
import ipywidgets   # from IPython.display import display; display(amp)

def H_W_split(input_str):
    H, W = input_str.split('H')[0], input_str.split('x')[1].split('W')[0]
    return int(H), int(W)


def update_plot(X,Y,block_size,nearest_10,block_type):

    # Hard code input, used only on if statement
    n_blocks = 4                # DO NOT CHANGE HERE
    
    if block_size=='mica/bkg':
        block_size = '25Hx25W'
        nearest_10 = False
    
    #---------- Computations ----------
    # Height Width extraction
    print('\n\n----------------------------------------')
    print('Block Size = ', block_size)
    H, W = H_W_split(block_size)            

    # Height Width to pixel conversion
    print('pixel_per_mm ', pixel_per_mm)
    inc_H = int(math.ceil(pixel_per_mm*H/1000/10.0)*10) if nearest_10==True else int(math.ceil(pixel_per_mm*H/1000))  # pixel_per_mm = 620, 620*300/1000 --> 300mm to 300 µm  pixel translation for Y cordinates
    inc_W = int(math.ceil(pixel_per_mm*W/1000/10.0)*10) if nearest_10==True else int(math.ceil(pixel_per_mm*W/1000))   # pixel_per_mm = 620, 620*300/1000 --> 300mm to 300 µm  pixel translation for X cordinates
    print(f'H = {H}µm --> pixel increment {pixel_per_mm*H/1000} rounded to ~{inc_H}, W = {W}µm --> pixel increment {pixel_per_mm*W/1000} rounded to ~{inc_W}')
    print('----------------------------------------')

    if block_type == 'one_block' :    
        # outputs list
        output_list = [[X, Y, X+inc_W, Y+inc_H]]
        print('>>> one big block\n',output_list)


    if block_type == 'multiple_block' and n_blocks==4:
        output_list = [[X, Y, X+inc_W, Y+inc_H], [X+inc_W, Y, X+inc_W*2, Y+inc_H], [X, Y+inc_H, X+inc_W, Y+inc_H*2], [X+inc_W, Y+inc_H, X+inc_W*2, Y+inc_H*2]]
        print('>>> multiple blocks\n',output_list)   
    
#---------- inputs ----------

options = ['300Hx300W', '600Hx600W', '600Hx300W', '300Hx600W', '300Hx900W', '900Hx300W', '50Hx50W', '25Hx25W', 'mica/bkg']       # change here  '16Hx16W' or mica
X = ipywidgets.IntSlider(min=0, max=12000, value=0, description = "X", layout=ipywidgets.Layout(width='50%'))
Y = ipywidgets.IntSlider(min=0, max=12000, value=0, description = "Y", layout=ipywidgets.Layout(width='50%'))
block_size = ipywidgets.Dropdown(options=options, value = options[0], description='Block Size : ', disabled=False)
nearest_10 = ipywidgets.Checkbox(value=True, description='Nearest 10 Rounding : ', disabled=False, indent=False )
block_type = ipywidgets.Dropdown(options=['one_block', 'multiple_block'], value = 'one_block', description='Block Type : ', disabled=False)

ipywidgets.interactive(update_plot, X=X, Y=Y, block_size=block_size, nearest_10=nearest_10, block_type=block_type)

interactive(children=(IntSlider(value=0, description='X', layout=Layout(width='50%'), max=12000), IntSlider(va…

In [28]:
#### run - 5th cell
# 50x50 scan timing
X_µ = 300
Y_µ = 600
beam_size_µ = 5
per_diff_opt_time = 0.8 # (0.5 exposure time + others)
mins = X_µ/beam_size_µ*Y_µ/beam_size_µ*per_diff_opt_time*60/3600  # 60/3600 to convernt to minute
print(mins)

96.0


In [13]:
[6225, 2750, 6585, 2930], 

192.0


In [97]:
### March-2021 Run

sdict1 = {"2428_EC": [[3232,628,3232+160,628+160], [3232+160,628,3232+320,628+160], [3232,628+160,3232+160,628+320],[3232+160,628+160,3232+320,628+320]], "2428_EC_mica":[[1630,1550,1630+16,1550+16]], \
          "2428_FC": [[1033,5932,1033+160,5932+160], [1033+160,5932,1033+320,5932+160], [1033,5932+160,1033+160,5932+320],[1033+160,5932+160,1033+320,5932+320]], "2428_FC_mica":[[4420,5760,4420+16,5760+16]], \
          "2040_EC": [[6230,2600,6230+160,2600+160], [6230+160,2600,6230+320,2600+160], [6230,2600+160,6230+160,2600+320],[6230+160,2600+160,6230+320,2600+320]], "2040_EC_mica":[[5600,500,5600+16,500+16]], \
          "2040_FC": [[6586,7635,6586+160,7635+160], [6586+160,7635,6586+320,7635+160], [6586,7635+160,6586+160,7635+320],[6586+160,7635+160,6586+320,7635+320]], "2040_FC_mica":[[6540,8200,6540+16,8200+16]], \
          "sample1_bkg": [[5000,2700,5000+16,2700+16]]}

sdict2 = {"2302_FC": [[2893,3135,2893+160,3135+160], [2893+160,3135,2893+320,3135+160], [2893,3135+160,2893+160,3135+320],[2893+160,3135+160,2893+320,3135+320]], "2302_FC_mica": [[4224,2915,4224+16,2915+16]], \
          "2302_EC": [[3880,7000,3880+160,7000+160], [3880+160,7000,3880+320,7000+160], [3880,7000+160,3880+160,7000+320],[3880+160,7000+160,3880+320,7000+320]], "2302_EC_mica": [[759,6050,759+16,6050+16]],\
          "2144_FC": [[7860,1832,7860+160,1832+160], [7860+160,1832,7860+320,1832+160], [7860,1832+160,7860+160,1832+320],[7860+160,1832+160,7860+320,1832+320]], "2144_FC_mica": [[5313,2000,5313+16,2000+16]],\
          "2144_EC": [[5786,7194,5786+160,7194+160], [5786+160,7194,5786+320,7194+160], [5786,7194+160,5786+160,7194+320],[5786+160,7194+160,5786+320,7194+320]], "2144_EC_mica": [[6611,7876,6611+16,7876+16]], \
          "sample2_bkg": [[4800,8300,4800+16,8300+16]]}

sdict3 = {"2017_FC": [[2740,2260,2740+160,2260+160], [2740+160,2260,2740+320,2260+160], [2740+320,2260,2740+480,2260+160],[2740+480,2260,2740+640,2260+160]],     "2302_FC_mica": [[3500,2500,3500+16,2500+16]], \
          "2017_EC": [[4120,6420,4120+160,6420+160], [4120+160,6420,4120+320,6420+160], [4120,6420+160,4120+160,6420+320],[4120+160,6420+160,4120+320,6420+320]], "2017_EC_mica": [[3500,8000,3500+16,8000+16]], \
          "2062_FC": [[8212,1640,8212+160,1640+160], [8212,1640+160,8212+160,1640+320], [8613,2090,8613+160,2090+160], [8613+160,2090,8613+320,2090+160]], "2062_FC_mica": [[8060,800,8060+16,800+16]], \
          "2062_EC": [[5310,7590,5310+160,7590+160], [5310+160,7590,5310+320,7590+160], [5310,7590+160,5310+160,7590+320],[5310+160,7590+160,5310+320,7590+320]], "2062_EC_mica": [[8701,5780,8701+16,5780+16]],\
          "sample3_bkg": [[5093,6094,5093+16,6094+16]]}

sdict41 = {"1887_FC": [[2010,2290,2010+160,2290+160]], "1887_FC_mica": [[4170,2465,4170+16,2465+16]], \
          "1887_EC": [[1250,6130,1250+160,6130+160]], "1887_EC_mica": [[1832,8230,1832+16,8230+16]], \
          "2018_EC": [[7950,7865,7950+160,7865+160], [7950+160,7865,7950+320,7865+160], [7950+320,7865,7950+480,7865+160],[7950+480,7865,7950+640,7865+160]],   "2018_EC_mica": [[7925,8300,7925+16,8300+16]], \
          "sample4_bkg": [[4800,2900,4800+16,2900+16]],}
sdict42 = {"2018_FC": [[7030,730,7030+160,730+160], [7030+160,730,7030+320,730+160], [7030,730+160,7030+160,730+320],[7030+160,730+160,7030+320,730+320]],       "2018_FC_mica": [[8950,1210,8950+16,1210+16]]}

sdict43 = {"2018_FC": [[7360,1040,7360+160,1040+160]]}

sdict5 = {"1886_FC": [[1100,1628,1100+160,1628+160]], "1886_FC_bkg": [[5665,1243,5665+16,1243+16]], "1886_FC_mica": [[4190,1960,4190+16,1960+16]],\
          "1886_EC": [[3921,7172,3921+160,7172+160]], "1886_EC_bkg": [[5665,6127,5665+16,6127+16]], "1886_EC_mica": [[3310,6140,3310+16,6140+16]],}

annotate_img("P1img.png", sdict1) # {**sdict3, **sdict4})
annotate_img("P2img.png", sdict2) # {**sdict3, **sdict4})
annotate_img("P3img.png", sdict3) # {**sdict3, **sdict4})
annotate_img("P4img.png", {**sdict41, **sdict42}) #  sdict5) # {**sdict3, **sdict4})
annotate_img("P4img.png", sdict43)    # another
annotate_img("P5img.png", sdict5) # {**sdict3, **sdict4})

In [97]:
### July-2021 Run

"""
sdict3 = {"rTgDI-1621": [[5614, 2188, 5672, 2264], [5464, 2384, 5556, 2508], [5394, 1644, 5482, 1700],
                         [5336, 1608, 5390, 1662], [5204, 1908, 5274, 1968], [5148, 1806, 5224, 1882],
                         [4994, 1950, 5124, 2048]],
          "rTgDI-1599": [[4508, 1206, 4576, 1266], [4402, 1434, 4460, 1588],
                         [4334, 984, 4412, 1090], [3978, 1488, 4038, 1612]]
         }
"""
sdict1 = {"2428_EC": [[3360,10612,3360+190,10612+190], [3360+190,10612,3360+380,10612+190], [3360,10612+190,3360+190,10612+380], [3360+190,10612+190,3360+380,10612+380]], "2428_EC_mica":[[740,9400,740+16,9400+16]], \
          "2428_EC-Spec": [[3880, 7460, 3880+190, 7460+190]], \
          "2428_FC": [[740, 4420, 740+190, 4420+190], [740+190, 4420, 740+380, 4420+190], [740, 4420+190, 740+190, 4420+380], [740+190, 4420+190, 740+380, 4420+380]], "2428_FC_mica": [[4960,4000,4960+16,4000+16]], \
          "2040_EC": [[6783, 8146+190, 6783+190, 8146+380]], "2040_EC_mica":[[10285,9053,10285+16,9053+16]], \
          "2040_FC": [[7271, 2277, 7271+380, 2277+380]], "2040_FC_mica": [[10304,4232,10304+16,4232+16]], \
          "sample1_bkg": [[5632,4752,5632+16,4752+16]]}

sdict7 = {"1593_EC": [[7935, 3747, 8125, 3937], [9280, 2776, 9470, 2966]], "1593_EC_mica": [[10088, 2960, 10104, 2976]] , \
          "1593_FC": [[9190, 8712, 9570, 8902]], "1593_FC_mica":  [[10042, 9030, 10058, 9046]], \
          "2109_EC": [[2232, 2528, 2422, 2908],  [2966, 3638, 3156, 3828] ], "2109_EC_mica":  [[3000, 2900, 3016, 2916]] , \
          "2109_FC":  [[2750, 9177, 2940, 9557]], "2109_FC_mica":  [[3126, 9315, 3142, 9331]] , \
          "sample7_bkg": [[4800, 9120, 4816, 9136]]}

sdict6 = {"2029_EC": [[7840, 1925, 8030, 2485]], "2029_EC_mica":  [[8500, 2056, 8516, 2072]] , \
          "2029_Dentate_gyrus":    [[8095, 10460, 8475, 10650], [8690, 9260, 8880, 9450]], "2029_dentate_gyrus_mica":  [[9500, 10460, 9516, 10476]] , \
          "2346_EC": [[860, 4140, 1240, 4520], [1240, 4140, 1620, 4520]] , "2346_EC_mica":  [[634, 4320, 650, 4336]]   , \
          "2346_Dentate_gyrus" :  [[3800, 10180, 4180, 10370], [3986, 9740, 4176, 9930]], "2346_dentate_gyrus_mica" : [[4218, 10518, 4234, 10534]] , \
          "sample6_bkg": [[5320, 9196, 5336, 9212]]}

sdict5 = {"2512_EC": [[8580, 3374, 8960, 3564], [9986, 3168, 10176, 3358]], "2512_EC_mica" : [[8066, 3374, 8082, 3390]], \
          "2512_Caudate": [[6816, 9333, 7006, 9523], [6886, 10000, 7266, 10190]], "2512_caudate_mica": [[6416, 9333, 6432, 9349]],\
          "2410_EC":   [[2510, 2880, 2700, 3070], [2000, 3140, 2190, 3330]], "2410_EC_mica": [[3000, 3216, 3016, 3232]] , \
          "2410_Caudate": [[3076, 8848, 3266, 9038]], "2410_caudate_mica": [[3076, 8048, 3092, 8064]] , \
          "sample5_bkg":  [[5520, 3040, 5536, 3056]]}

sdict4 = {"2423_Nigra" : [[1993, 3450, 2373, 3830], [2373, 3450, 2753, 3830], [1718, 4120, 1908, 4500], [3066, 3320, 3256, 3510]], "2423_Nigra_mica" : [[3936, 4074, 3952, 4090]] , \
          "2423_Pallidus" : [[1661, 9042, 2041, 9422]] , "2423_Pallidus_mica" : [[654, 10631, 670, 10647]], \
          "2525_Occipital" : [[8030, 4380, 8220, 4570]], "2525_occipital_mica" : [[7449, 4800, 7465, 4816]] , \
          "2525_Parietal" : [[7900, 9090, 8090, 9280]], "2525_Parietal_mica" : [[7794, 9538, 7810, 9554]] ,\
          "sample4_bkg": [[5520, 3040, 5536, 3056]]}

sdict2 = {"1892_Sarkosyl_Tau" :      [[7722, 3920, 7762, 3960] , [8464, 3936, 8504, 3976]] , \
          "1892_High_Molecular_Tau": [[7843, 8784, 7883, 8824] , [8295, 8750, 8335, 8790]] , \
          "Recombinant_Tau":         [[3377, 3773, 3417, 3813] , [2520, 4147, 2560, 4187]] , \
          "sample2_bkg" :            [[7876, 4686, 7892, 4702]]}

sdict8 = {"2512_EC_SiN" : [[7775, 8333, 8155, 8713]] , "2512_EC_SiN_SiN" : [[8448, 9178, 8464, 9194]], \
          "sample8_bkg" : [[5020, 8160, 5036, 8176]] }

sdict3 = {"1892_EC" : [[9157, 3540, 9537, 3730]] , "1892_EC_mica" : [[9688, 3760, 9704, 3776]] ,\
          "1892_FC" : [[9657, 9849, 9847, 10039], [10146, 9350, 10336, 9540]] , "1892_FC_mica" : [[9900, 10420, 9916, 10436]]   , \
          "sample3_bkg" : [[5552, 8592, 5568, 8608]]}

sdict9 = {"2346_EC_SiN" :  [[7260, 8336, 7640, 8716]] , "2346_EC_SiN_SiN" : [[8640, 9520, 8656, 9536]] , \
          "2346_Dentate_gyrus_SiN" : [[7100, 3420, 7480, 3610],  [7078, 4328, 7458, 4518]], "2346_Dentate_gyrus_SiN_SiN" : [[6986, 3412, 7002, 3428]] ,\
          "2029_Dentate_SiN" :  [[2300, 4460, 2680, 4840]], "2029_Dentate_SiN_SiN" : [[3102, 4433, 3118, 4449]] ,\
          "2029_EC_SiN" : [[2480, 8210, 2860, 8590]] , "2029_EC_SiN_SiN" :  [[2992, 9064, 3008, 9080]] ,\
          "sample9_bkg" : [[5544, 8151, 5560, 8167]]}

# annotate_img("imgH1.png", sdict1) # {**sdict3, **sdict4})
# annotate_img("imgH7.png", sdict7) # {**sdict3, **sdict4})
# annotate_img("imgH6.png", sdict6) # {**sdict3, **sdict4})
# annotate_img("imgH5.png", sdict5) # {**sdict3, **sdict4})
# annotate_img("imgH4.png", sdict4) # {**sdict3, **sdict4})
# annotate_img("imgH3.png", sdict3) # {**sdict3, **sdict4})   
# annotate_img("imgH2.png", sdict2) # {**sdict3, **sdict4}) 
# annotate_img("imgH8.png", sdict8) # {**sdict3, **sdict4}) 
annotate_img("imgH9.png", sdict9) # {**sdict3, **sdict4})   

In [80]:
from essential_func import *
import math 

# input
dict_input = sdict3

# computation
roi_dict = {}
for key in dict_input.keys():
    roi_list = []
    for value in dict_input[key]:
        print
        roi_list.append((value[3] - value[1] , value[2] - value[0]))
    roi_dict[key] = roi_list                  # dictionary with all the differences

diff_types = flatten([values for values in roi_dict.values()])
unique, frequency = np.unique(diff_types, return_counts = True)

# print unique and frequency values array
print("Unique Values:", unique , "\nFrequency Values:", frequency)

for i in unique:
    print(i, 'x--> and y-->', math.ceil(i/pixel_per_mm/5*1000))

190 380
16 16
190 190
190 190
16 16
16 16
Unique Values: [ 16 190 380] 
Frequency Values: [3 2 1]
16 x--> and y--> 6
190 x--> and y--> 62
380 x--> and y--> 123


In [81]:
[(16,16), 36], [(40,40), 14*14], [(190,190), 62*62], [(380, 380), 123*124]

([(16, 16), 36], [(40, 40), 196], [(190, 190), 3844], [(380, 380), 15252])

### 